# **Swin Transformer pretrained**

In [ ]:
import os
import pandas as pd
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision.models as models
import numpy as np
import timm
import torch
import torch.nn.functional as F
import numpy as np
from torcheval.metrics import MulticlassAccuracy, MulticlassPrecision, MulticlassRecall, MulticlassF1Score, MulticlassAUROC, MulticlassAUPRC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
import random


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)  # For CPU
    torch.cuda.manual_seed(seed)  # For GPU (if used)
    torch.cuda.manual_seed_all(seed)  # For all GPUs
    torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior
    torch.backends.cudnn.benchmark = False


def calculate_metrics(predicted, probabilities, labels, num_classes):
    # Initialize Torcheval metrics
    accuracy_metric = MulticlassAccuracy(num_classes=num_classes, average="macro")
    precision_metric = MulticlassPrecision(num_classes=num_classes, average="macro")
    recall_metric = MulticlassRecall(num_classes=num_classes, average="macro")
    f1_score_metric = MulticlassF1Score(num_classes=num_classes, average="macro")
    auc_metric = MulticlassAUROC(num_classes=num_classes, average="macro")
    aupr_metric = MulticlassAUPRC(num_classes=num_classes, average="macro")

    # Update metrics with predictions
    accuracy_metric.update(predicted, labels)
    precision_metric.update(predicted, labels)
    recall_metric.update(predicted, labels)
    f1_score_metric.update(predicted, labels)
    auc_metric.update(probabilities, labels)
    aupr_metric.update(probabilities, labels)

    # Compute metrics
    accuracy = accuracy_metric.compute()
    precision = precision_metric.compute()
    recall = recall_metric.compute()
    f1_score = f1_score_metric.compute()
    auc = auc_metric.compute()
    aupr = aupr_metric.compute()

    # Specificity calculation
    conf_matrix = confusion_matrix(labels.cpu().numpy(), predicted.cpu().numpy(), labels=range(num_classes))
    specificity_per_class = []

    for i in range(num_classes):
        TP = conf_matrix[i, i]
        FP = conf_matrix[:, i].sum() - TP
        FN = conf_matrix[i, :].sum() - TP
        TN = conf_matrix.sum() - (TP + FP + FN)

        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
        specificity_per_class.append(specificity)

    macro_specificity = np.mean(specificity_per_class)

    # MCC calculation for multi-class classification
    mcc_per_class = []
    for i in range(num_classes):
        # Treat each class as the positive class and others as negative
        binary_pred = (predicted == i).int()
        binary_labels = (labels == i).int()
        mcc = matthews_corrcoef(binary_labels.cpu().numpy(), binary_pred.cpu().numpy())
        mcc_per_class.append(mcc)

    macro_mcc = np.mean(mcc_per_class)

    # Print all metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")
    print(f"AUC: {auc:.4f}")
    print(f"AUPR: {aupr:.4f}")
    print(f"Specificity per class: {specificity_per_class}")
    print(f"Macro-average Specificity: {macro_specificity:.4f}")
    print(f"MCC per class: {mcc_per_class}")
    print(f"Macro-average MCC: {macro_mcc:.4f}")

    f1_score_metric_micro = MulticlassF1Score(num_classes=num_classes, average="micro")
    f1_score_metric_micro.update(predicted, labels)
    f1_score_micro = f1_score_metric_micro.compute()
    print(f"F1 Score (micro): {f1_score_micro:.4f}")

    f1_score_metric_weighted = MulticlassF1Score(num_classes=num_classes, average="weighted")
    f1_score_metric_weighted.update(predicted, labels)
    f1_score_weighted = f1_score_metric_weighted.compute()
    print(f"F1 Score (weighted): {f1_score_weighted:.4f}")


    # Return metrics as a dictionary
    return {
        "accuracy": "{:.4f}".format(accuracy.item()),
        "precision": "{:.4f}".format(precision.item()),
        "recall": "{:.4f}".format(recall.item()),
        "f1_score": "{:.4f}".format(f1_score.item()),
        "auc": "{:.4f}".format(auc.item()),
        "aupr": "{:.4f}".format(aupr.item()),
        "macro_specificity": "{:.4f}".format(macro_specificity.item()),
        "macro_mcc": "{:.4f}".format(macro_mcc.item())
    }


class ImageDataset(Dataset):
    def __init__(self, images, features, labels, transform=None):
        self.images = images
        self.features = features
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        feature = self.features[idx]
        label = self.labels[idx]
        return image, feature, label


# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

set_seed(42)

dataset_path = "baldder_tissue_classification"

train_df = pd.read_csv("train_df.csv")
val_df = pd.read_csv("val_df.csv")

train_images = []
train_labels = []
for idx, row in train_df.iterrows():
    image_path = os.path.join(dataset_path, row[1], row[0])
    image = Image.open(image_path)
    image = transform(image)
    train_images.append(image)
    train_labels.append(row['encoded_label'])

loaded_images = np.load("top_images.npy")
loaded_labels = np.load("top_labels.npy")
print(len(train_images))
for i in range(len(loaded_images)):
    image = loaded_images[i]
    label = loaded_labels[i]

    # Denormalize the image
    image = (image * np.array([0.229, 0.224, 0.225])[:, None, None]) + np.array([0.485, 0.456, 0.406])[:, None, None]
    image = (image * 255).astype(np.uint8)  # Convert to integer type for PIL compatibility

    # Convert NumPy array to PIL image
    image = Image.fromarray(image.transpose(1, 2, 0))  # H x W x C format

    # Apply transformations
    image = transform(image)

    # Convert back to PyTorch tensor (if required)
    image = torch.tensor(image, dtype=torch.float32)
    train_images.append(image)
    train_labels.append(label)

print(len(train_images))

val_images = []
val_labels = []
for idx, row in val_df.iterrows():
    image_path = os.path.join(dataset_path, row[1], row[0])
    image = Image.open(image_path)
    image = transform(image)
    val_images.append(image)
    val_labels.append(row['encoded_label'])

print(len(val_images))

train_features = np.load("features_RFE.npy")[:-val_df.shape[0]]
augmented_features = np.load("features_top_images_RFE.npy")
train_features = np.concatenate((train_features, augmented_features), axis=0)
val_features = np.load("features_RFE.npy")[-val_df.shape[0]:]

print(train_features.shape)
print(val_features.shape)

# Create DataLoaders
train_dataset = ImageDataset(train_images, train_features, train_labels, transform=transform)
val_dataset = ImageDataset(val_images, val_features, val_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.block = timm.create_model('swin_base_patch4_window7_224.ms_in22k_ft_in1k', pretrained=True)
        # self.block.head = nn.Linear(self.block.num_features, 1000)

        self.fc1 = nn.Linear(1000, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)

        self.fc_ann1 = nn.Linear(512, 256)
        self.bn_ann1 = nn.BatchNorm1d(256)
        self.relu_ann1 = nn.ReLU()
        self.dropout_ann1 = nn.Dropout(0.2)
        self.fc_ann2 = nn.Linear(256, 128)
        self.bn_ann2 = nn.BatchNorm1d(128)
        self.relu_ann2 = nn.ReLU()
        self.dropout_ann2 = nn.Dropout(0.2)

        self.fc_comb = nn.Linear(384, 256)
        self.bn_comb = nn.BatchNorm1d(256)
        self.relu_comb = nn.ReLU()
        self.dropout_comb = nn.Dropout(0.2)

        self.fc3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.2)
        self.fc4 = nn.Linear(128, num_classes)

        self.relu = nn.ReLU()


    def forward(self, x, features):
        features = features.float()
        x = self.block(x)
        # print(x.shape)
        x = x.view(-1, 1000)  # Flatten
        x = self.dropout1(self.relu1(self.bn1(self.fc1(x))))
        x = self.dropout2(self.relu2(self.bn2(self.fc2(x))))

        features = self.dropout_ann1(self.relu_ann1(self.bn_ann1(self.fc_ann1(features))))
        features = self.dropout_ann2(self.relu_ann2(self.bn_ann2(self.fc_ann2(features))))

        x = torch.cat((x, features), dim=1)
        x = self.dropout_comb(self.relu_comb(self.bn_comb(self.fc_comb(x))))

        x = self.dropout3(self.relu3(self.bn3(self.fc3(x))))
        x = self.fc4(x)
        return x


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
# Instantiate model, loss function, and optimizer
num_classes = 4
model = SimpleCNN(num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 50
all_metrics = []

best_f1 = -100
best_epoch = 0
best_metric = 0
with open('Results/swin_base_patch4_window7_224_pretrained.csv', 'w') as f:
    f.write("Epoch,Train Loss,Val Loss,Accuracy,Precision,Recall,F1 Score,AUC,AUPR,Specificity,MCC\n")
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for images, features, labels in train_loader:
            images, features, labels = images.to(device), features.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images, features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        train_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}")

        # Validation loop
        model.eval()
        running_loss = 0.0
        with torch.no_grad():
            y_pred = []
            y_true = []
            y_proba = []
            for images, features, labels in val_loader:
                images, features, labels = images.to(device), features.to(device), labels.to(device)

                outputs = model(images, features)
                loss = criterion(outputs, labels)

                running_loss += loss.item() * images.size(0)

                # Apply softmax to get probabilities
                probabilities = F.softmax(outputs, dim=1)

                # Get the predicted class and corresponding probability
                proba, predicted = torch.max(probabilities, 1)

                # Store results
                y_pred.extend(predicted.cpu().tolist())
                y_true.extend(labels.cpu().tolist())
                y_proba.extend(probabilities.cpu().tolist())

            val_loss = running_loss / len(val_loader.dataset)
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {val_loss:.4f}")
            # Convert lists to tensors
            y_pred = torch.tensor(y_pred)
            y_true = torch.tensor(y_true)
            y_proba = torch.tensor(y_proba)

            metrics = calculate_metrics(y_pred, y_proba, y_true, num_classes)
            all_metrics.append(metrics)

            if best_f1 < float(metrics["f1_score"]):
                best_f1 = float(metrics["f1_score"])
                best_epoch = epoch+1
                best_metric = metrics
            print('---------------------------------------------------------------------------------------------------')

            f.write(f"{epoch+1},{train_loss},{val_loss},{metrics['accuracy']},{metrics['precision']},{metrics['recall']},{metrics['f1_score']},{metrics['auc']},{metrics['aupr']},{metrics['macro_specificity']},{metrics['macro_mcc']}\n")
            f.flush()
    print("Training and validating complete")

<ipython-input-7-ebc52e334170>:155: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image_path = os.path.join(dataset_path, row[1], row[0])


1262


<ipython-input-7-ebc52e334170>:179: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  image = torch.tensor(image, dtype=torch.float32)


1762


<ipython-input-7-ebc52e334170>:188: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image_path = os.path.join(dataset_path, row[1], row[0])


316
(1762, 512)
(316, 512)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

Epoch 1/50, Loss: 1.0234
Epoch 1/50, Loss: 0.7297
Accuracy: 0.6266
Precision: 0.7177
Recall: 0.6266
F1 Score: 0.6278
AUC: 0.9193
AUPR: 0.7503
Specificity per class: [0.8362068965517241, 0.8391959798994975, 0.9955555555555555, 0.9931506849315068]
Macro-average Specificity: 0.9160
MCC per class: [0.5059248958578667, 0.7879623642614569, 0.8219586442409945, 0.18124266959083374]
Macro-average MCC: 0.5743
F1 Score (micro): 0.7690
F1 Score (weighted): 0.7500
---------------------------------------------------------------------------------------------------
Epoch 2/50, Loss: 0.9549
Epoch 2/50, Loss: 0.6845
Accuracy: 0.6649
Precision: 0.7315
Recall: 0.6649
F1 Score: 0.6681
AUC: 0.9124
AUPR: 0.7611
Specificity per class: [0.7413793103448276, 0.9447236180904522, 0.9955555555555555, 0.9828767123287672]
Macro-average Specificity: 0.9161
MCC per class: [0.5866148796725079, 0.6563418101663109, 0.8994303870852426, 0.2893656919124181]
Macro-average MCC: 0.6079
F1 Score (micro): 0.7563
F1 Score (weighte

# Inception v3 pretrained

In [ ]:
import os
import pandas as pd
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision.models as models
import numpy as np
import timm
import torch
import torch.nn.functional as F
import numpy as np
from torcheval.metrics import MulticlassAccuracy, MulticlassPrecision, MulticlassRecall, MulticlassF1Score, MulticlassAUROC, MulticlassAUPRC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
import random


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)  # For CPU
    torch.cuda.manual_seed(seed)  # For GPU (if used)
    torch.cuda.manual_seed_all(seed)  # For all GPUs
    torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior
    torch.backends.cudnn.benchmark = False


def calculate_metrics(predicted, probabilities, labels, num_classes):
    # Initialize Torcheval metrics
    accuracy_metric = MulticlassAccuracy(num_classes=num_classes, average="macro")
    precision_metric = MulticlassPrecision(num_classes=num_classes, average="macro")
    recall_metric = MulticlassRecall(num_classes=num_classes, average="macro")
    f1_score_metric = MulticlassF1Score(num_classes=num_classes, average="macro")
    auc_metric = MulticlassAUROC(num_classes=num_classes, average="macro")
    aupr_metric = MulticlassAUPRC(num_classes=num_classes, average="macro")

    # Update metrics with predictions
    accuracy_metric.update(predicted, labels)
    precision_metric.update(predicted, labels)
    recall_metric.update(predicted, labels)
    f1_score_metric.update(predicted, labels)
    auc_metric.update(probabilities, labels)
    aupr_metric.update(probabilities, labels)

    # Compute metrics
    accuracy = accuracy_metric.compute()
    precision = precision_metric.compute()
    recall = recall_metric.compute()
    f1_score = f1_score_metric.compute()
    auc = auc_metric.compute()
    aupr = aupr_metric.compute()

    # Specificity calculation
    conf_matrix = confusion_matrix(labels.cpu().numpy(), predicted.cpu().numpy(), labels=range(num_classes))
    specificity_per_class = []

    for i in range(num_classes):
        TP = conf_matrix[i, i]
        FP = conf_matrix[:, i].sum() - TP
        FN = conf_matrix[i, :].sum() - TP
        TN = conf_matrix.sum() - (TP + FP + FN)

        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
        specificity_per_class.append(specificity)

    macro_specificity = np.mean(specificity_per_class)

    # MCC calculation for multi-class classification
    mcc_per_class = []
    for i in range(num_classes):
        # Treat each class as the positive class and others as negative
        binary_pred = (predicted == i).int()
        binary_labels = (labels == i).int()
        mcc = matthews_corrcoef(binary_labels.cpu().numpy(), binary_pred.cpu().numpy())
        mcc_per_class.append(mcc)

    macro_mcc = np.mean(mcc_per_class)

    # Print all metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")
    print(f"AUC: {auc:.4f}")
    print(f"AUPR: {aupr:.4f}")
    print(f"Specificity per class: {specificity_per_class}")
    print(f"Macro-average Specificity: {macro_specificity:.4f}")
    print(f"MCC per class: {mcc_per_class}")
    print(f"Macro-average MCC: {macro_mcc:.4f}")

    f1_score_metric_micro = MulticlassF1Score(num_classes=num_classes, average="micro")
    f1_score_metric_micro.update(predicted, labels)
    f1_score_micro = f1_score_metric_micro.compute()
    print(f"F1 Score (micro): {f1_score_micro:.4f}")

    f1_score_metric_weighted = MulticlassF1Score(num_classes=num_classes, average="weighted")
    f1_score_metric_weighted.update(predicted, labels)
    f1_score_weighted = f1_score_metric_weighted.compute()
    print(f"F1 Score (weighted): {f1_score_weighted:.4f}")


    # Return metrics as a dictionary
    return {
        "accuracy": "{:.4f}".format(accuracy.item()),
        "precision": "{:.4f}".format(precision.item()),
        "recall": "{:.4f}".format(recall.item()),
        "f1_score": "{:.4f}".format(f1_score.item()),
        "auc": "{:.4f}".format(auc.item()),
        "aupr": "{:.4f}".format(aupr.item()),
        "macro_specificity": "{:.4f}".format(macro_specificity.item()),
        "macro_mcc": "{:.4f}".format(macro_mcc.item())
    }


class ImageDataset(Dataset):
    def __init__(self, images, features, labels, transform=None):
        self.images = images
        self.features = features
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        feature = self.features[idx]
        label = self.labels[idx]
        return image, feature, label


# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

set_seed(42)

dataset_path = "baldder_tissue_classification"

train_df = pd.read_csv("train_df.csv")
val_df = pd.read_csv("val_df.csv")

train_images = []
train_labels = []
for idx, row in train_df.iterrows():
    image_path = os.path.join(dataset_path, row[1], row[0])
    image = Image.open(image_path)
    image = transform(image)
    train_images.append(image)
    train_labels.append(row['encoded_label'])

loaded_images = np.load("top_images.npy")
loaded_labels = np.load("top_labels.npy")
print(len(train_images))
for i in range(len(loaded_images)):
    image = loaded_images[i]
    label = loaded_labels[i]

    # Denormalize the image
    image = (image * np.array([0.229, 0.224, 0.225])[:, None, None]) + np.array([0.485, 0.456, 0.406])[:, None, None]
    image = (image * 255).astype(np.uint8)  # Convert to integer type for PIL compatibility

    # Convert NumPy array to PIL image
    image = Image.fromarray(image.transpose(1, 2, 0))  # H x W x C format

    # Apply transformations
    image = transform(image)

    # Convert back to PyTorch tensor (if required)
    image = torch.tensor(image, dtype=torch.float32)
    train_images.append(image)
    train_labels.append(label)

print(len(train_images))

val_images = []
val_labels = []
for idx, row in val_df.iterrows():
    image_path = os.path.join(dataset_path, row[1], row[0])
    image = Image.open(image_path)
    image = transform(image)
    val_images.append(image)
    val_labels.append(row['encoded_label'])

print(len(val_images))

train_features = np.load("features_RFE.npy")[:-val_df.shape[0]]
augmented_features = np.load("features_top_images_RFE.npy")
train_features = np.concatenate((train_features, augmented_features), axis=0)
val_features = np.load("features_RFE.npy")[-val_df.shape[0]:]

print(train_features.shape)
print(val_features.shape)

# Create DataLoaders
train_dataset = ImageDataset(train_images, train_features, train_labels, transform=transform)
val_dataset = ImageDataset(val_images, val_features, val_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.block = timm.create_model('inception_v3.tv_in1k', pretrained=True)
        # self.block.head = nn.Linear(self.block.num_features, 1000)

        self.fc1 = nn.Linear(1000, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)

        self.fc_ann1 = nn.Linear(512, 256)
        self.bn_ann1 = nn.BatchNorm1d(256)
        self.relu_ann1 = nn.ReLU()
        self.dropout_ann1 = nn.Dropout(0.2)
        self.fc_ann2 = nn.Linear(256, 128)
        self.bn_ann2 = nn.BatchNorm1d(128)
        self.relu_ann2 = nn.ReLU()
        self.dropout_ann2 = nn.Dropout(0.2)

        self.fc_comb = nn.Linear(384, 256)
        self.bn_comb = nn.BatchNorm1d(256)
        self.relu_comb = nn.ReLU()
        self.dropout_comb = nn.Dropout(0.2)

        self.fc3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.2)
        self.fc4 = nn.Linear(128, num_classes)

        self.relu = nn.ReLU()


    def forward(self, x, features):
        features = features.float()
        x = self.block(x)
        # print(x.shape)
        x = x.view(-1, 1000)  # Flatten
        x = self.dropout1(self.relu1(self.bn1(self.fc1(x))))
        x = self.dropout2(self.relu2(self.bn2(self.fc2(x))))

        features = self.dropout_ann1(self.relu_ann1(self.bn_ann1(self.fc_ann1(features))))
        features = self.dropout_ann2(self.relu_ann2(self.bn_ann2(self.fc_ann2(features))))

        x = torch.cat((x, features), dim=1)
        x = self.dropout_comb(self.relu_comb(self.bn_comb(self.fc_comb(x))))

        x = self.dropout3(self.relu3(self.bn3(self.fc3(x))))
        x = self.fc4(x)
        return x


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
# Instantiate model, loss function, and optimizer
num_classes = 4
model = SimpleCNN(num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 50
all_metrics = []

best_f1 = -100
best_epoch = 0
best_metric = 0
with open('Results/inception_v3_pretrained.csv', 'w') as f:
    f.write("Epoch,Train Loss,Val Loss,Accuracy,Precision,Recall,F1 Score,AUC,AUPR,Specificity,MCC\n")
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for images, features, labels in train_loader:
            images, features, labels = images.to(device), features.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images, features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        train_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}")

        # Validation loop
        model.eval()
        running_loss = 0.0
        with torch.no_grad():
            y_pred = []
            y_true = []
            y_proba = []
            for images, features, labels in val_loader:
                images, features, labels = images.to(device), features.to(device), labels.to(device)

                outputs = model(images, features)
                loss = criterion(outputs, labels)

                running_loss += loss.item() * images.size(0)

                # Apply softmax to get probabilities
                probabilities = F.softmax(outputs, dim=1)

                # Get the predicted class and corresponding probability
                proba, predicted = torch.max(probabilities, 1)

                # Store results
                y_pred.extend(predicted.cpu().tolist())
                y_true.extend(labels.cpu().tolist())
                y_proba.extend(probabilities.cpu().tolist())

            val_loss = running_loss / len(val_loader.dataset)
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {val_loss:.4f}")
            # Convert lists to tensors
            y_pred = torch.tensor(y_pred)
            y_true = torch.tensor(y_true)
            y_proba = torch.tensor(y_proba)

            metrics = calculate_metrics(y_pred, y_proba, y_true, num_classes)
            all_metrics.append(metrics)

            if best_f1 < float(metrics["f1_score"]):
                best_f1 = float(metrics["f1_score"])
                best_epoch = epoch+1
                best_metric = metrics
            print('---------------------------------------------------------------------------------------------------')

            f.write(f"{epoch+1},{train_loss},{val_loss},{metrics['accuracy']},{metrics['precision']},{metrics['recall']},{metrics['f1_score']},{metrics['auc']},{metrics['aupr']},{metrics['macro_specificity']},{metrics['macro_mcc']}\n")
            f.flush()
    print("Training and validating complete")

<ipython-input-12-1143e10287fd>:155: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image_path = os.path.join(dataset_path, row[1], row[0])


1262


<ipython-input-12-1143e10287fd>:179: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  image = torch.tensor(image, dtype=torch.float32)


1762


<ipython-input-12-1143e10287fd>:188: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image_path = os.path.join(dataset_path, row[1], row[0])


316
(1762, 512)
(316, 512)


model.safetensors:   0%|          | 0.00/95.5M [00:00<?, ?B/s]

Epoch 1/50, Loss: 1.1622
Epoch 1/50, Loss: 0.6979
Accuracy: 0.7953
Precision: 0.8196
Recall: 0.7953
F1 Score: 0.8042
AUC: 0.9674
AUPR: 0.8698
Specificity per class: [0.9612068965517241, 0.8944723618090452, 0.9777777777777777, 0.9726027397260274]
Macro-average Specificity: 0.9515
MCC per class: [0.7312652394852064, 0.833055200914143, 0.930803504791323, 0.5469745005202838]
Macro-average MCC: 0.7605
F1 Score (micro): 0.8639
F1 Score (weighted): 0.8600
---------------------------------------------------------------------------------------------------
Epoch 2/50, Loss: 0.7544
Epoch 2/50, Loss: 0.5043
Accuracy: 0.7648
Precision: 0.8746
Recall: 0.7648
F1 Score: 0.7698
AUC: 0.9690
AUPR: 0.8778
Specificity per class: [0.9224137931034483, 0.9899497487437185, 0.9377777777777778, 0.9965753424657534]
Macro-average Specificity: 0.9617
MCC per class: [0.8445252492384426, 0.9054549279550154, 0.8931165071612532, 0.3977415127859719]
Macro-average MCC: 0.7602
F1 Score (micro): 0.8892
F1 Score (weighted):

# resnet50 pretrained

In [ ]:
import os
import pandas as pd
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision.models as models
import numpy as np
import timm
import torch
import torch.nn.functional as F
import numpy as np
from torcheval.metrics import MulticlassAccuracy, MulticlassPrecision, MulticlassRecall, MulticlassF1Score, MulticlassAUROC, MulticlassAUPRC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
import random


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)  # For CPU
    torch.cuda.manual_seed(seed)  # For GPU (if used)
    torch.cuda.manual_seed_all(seed)  # For all GPUs
    torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior
    torch.backends.cudnn.benchmark = False


def calculate_metrics(predicted, probabilities, labels, num_classes):
    # Initialize Torcheval metrics
    accuracy_metric = MulticlassAccuracy(num_classes=num_classes, average="macro")
    precision_metric = MulticlassPrecision(num_classes=num_classes, average="macro")
    recall_metric = MulticlassRecall(num_classes=num_classes, average="macro")
    f1_score_metric = MulticlassF1Score(num_classes=num_classes, average="macro")
    auc_metric = MulticlassAUROC(num_classes=num_classes, average="macro")
    aupr_metric = MulticlassAUPRC(num_classes=num_classes, average="macro")

    # Update metrics with predictions
    accuracy_metric.update(predicted, labels)
    precision_metric.update(predicted, labels)
    recall_metric.update(predicted, labels)
    f1_score_metric.update(predicted, labels)
    auc_metric.update(probabilities, labels)
    aupr_metric.update(probabilities, labels)

    # Compute metrics
    accuracy = accuracy_metric.compute()
    precision = precision_metric.compute()
    recall = recall_metric.compute()
    f1_score = f1_score_metric.compute()
    auc = auc_metric.compute()
    aupr = aupr_metric.compute()

    # Specificity calculation
    conf_matrix = confusion_matrix(labels.cpu().numpy(), predicted.cpu().numpy(), labels=range(num_classes))
    specificity_per_class = []

    for i in range(num_classes):
        TP = conf_matrix[i, i]
        FP = conf_matrix[:, i].sum() - TP
        FN = conf_matrix[i, :].sum() - TP
        TN = conf_matrix.sum() - (TP + FP + FN)

        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
        specificity_per_class.append(specificity)

    macro_specificity = np.mean(specificity_per_class)

    # MCC calculation for multi-class classification
    mcc_per_class = []
    for i in range(num_classes):
        # Treat each class as the positive class and others as negative
        binary_pred = (predicted == i).int()
        binary_labels = (labels == i).int()
        mcc = matthews_corrcoef(binary_labels.cpu().numpy(), binary_pred.cpu().numpy())
        mcc_per_class.append(mcc)

    macro_mcc = np.mean(mcc_per_class)

    # Print all metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")
    print(f"AUC: {auc:.4f}")
    print(f"AUPR: {aupr:.4f}")
    print(f"Specificity per class: {specificity_per_class}")
    print(f"Macro-average Specificity: {macro_specificity:.4f}")
    print(f"MCC per class: {mcc_per_class}")
    print(f"Macro-average MCC: {macro_mcc:.4f}")

    f1_score_metric_micro = MulticlassF1Score(num_classes=num_classes, average="micro")
    f1_score_metric_micro.update(predicted, labels)
    f1_score_micro = f1_score_metric_micro.compute()
    print(f"F1 Score (micro): {f1_score_micro:.4f}")

    f1_score_metric_weighted = MulticlassF1Score(num_classes=num_classes, average="weighted")
    f1_score_metric_weighted.update(predicted, labels)
    f1_score_weighted = f1_score_metric_weighted.compute()
    print(f"F1 Score (weighted): {f1_score_weighted:.4f}")


    # Return metrics as a dictionary
    return {
        "accuracy": "{:.4f}".format(accuracy.item()),
        "precision": "{:.4f}".format(precision.item()),
        "recall": "{:.4f}".format(recall.item()),
        "f1_score": "{:.4f}".format(f1_score.item()),
        "auc": "{:.4f}".format(auc.item()),
        "aupr": "{:.4f}".format(aupr.item()),
        "macro_specificity": "{:.4f}".format(macro_specificity.item()),
        "macro_mcc": "{:.4f}".format(macro_mcc.item())
    }


class ImageDataset(Dataset):
    def __init__(self, images, features, labels, transform=None):
        self.images = images
        self.features = features
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        feature = self.features[idx]
        label = self.labels[idx]
        return image, feature, label


# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

set_seed(42)

dataset_path = "baldder_tissue_classification"

train_df = pd.read_csv("train_df.csv")
val_df = pd.read_csv("val_df.csv")

train_images = []
train_labels = []
for idx, row in train_df.iterrows():
    image_path = os.path.join(dataset_path, row[1], row[0])
    image = Image.open(image_path)
    image = transform(image)
    train_images.append(image)
    train_labels.append(row['encoded_label'])

loaded_images = np.load("top_images.npy")
loaded_labels = np.load("top_labels.npy")
print(len(train_images))
for i in range(len(loaded_images)):
    image = loaded_images[i]
    label = loaded_labels[i]

    # Denormalize the image
    image = (image * np.array([0.229, 0.224, 0.225])[:, None, None]) + np.array([0.485, 0.456, 0.406])[:, None, None]
    image = (image * 255).astype(np.uint8)  # Convert to integer type for PIL compatibility

    # Convert NumPy array to PIL image
    image = Image.fromarray(image.transpose(1, 2, 0))  # H x W x C format

    # Apply transformations
    image = transform(image)

    # Convert back to PyTorch tensor (if required)
    image = torch.tensor(image, dtype=torch.float32)
    train_images.append(image)
    train_labels.append(label)

print(len(train_images))

val_images = []
val_labels = []
for idx, row in val_df.iterrows():
    image_path = os.path.join(dataset_path, row[1], row[0])
    image = Image.open(image_path)
    image = transform(image)
    val_images.append(image)
    val_labels.append(row['encoded_label'])

print(len(val_images))

train_features = np.load("features_RFE.npy")[:-val_df.shape[0]]
augmented_features = np.load("features_top_images_RFE.npy")
train_features = np.concatenate((train_features, augmented_features), axis=0)
val_features = np.load("features_RFE.npy")[-val_df.shape[0]:]

print(train_features.shape)
print(val_features.shape)

# Create DataLoaders
train_dataset = ImageDataset(train_images, train_features, train_labels, transform=transform)
val_dataset = ImageDataset(val_images, val_features, val_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.block = timm.create_model('resnet50.a1_in1k', pretrained=True)
        # self.block.head = nn.Linear(self.block.num_features, 1000)

        self.fc1 = nn.Linear(1000, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)

        self.fc_ann1 = nn.Linear(512, 256)
        self.bn_ann1 = nn.BatchNorm1d(256)
        self.relu_ann1 = nn.ReLU()
        self.dropout_ann1 = nn.Dropout(0.2)
        self.fc_ann2 = nn.Linear(256, 128)
        self.bn_ann2 = nn.BatchNorm1d(128)
        self.relu_ann2 = nn.ReLU()
        self.dropout_ann2 = nn.Dropout(0.2)

        self.fc_comb = nn.Linear(384, 256)
        self.bn_comb = nn.BatchNorm1d(256)
        self.relu_comb = nn.ReLU()
        self.dropout_comb = nn.Dropout(0.2)

        self.fc3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.2)
        self.fc4 = nn.Linear(128, num_classes)

        self.relu = nn.ReLU()


    def forward(self, x, features):
        features = features.float()
        x = self.block(x)
        # print(x.shape)
        x = x.view(-1, 1000)  # Flatten
        x = self.dropout1(self.relu1(self.bn1(self.fc1(x))))
        x = self.dropout2(self.relu2(self.bn2(self.fc2(x))))

        features = self.dropout_ann1(self.relu_ann1(self.bn_ann1(self.fc_ann1(features))))
        features = self.dropout_ann2(self.relu_ann2(self.bn_ann2(self.fc_ann2(features))))

        x = torch.cat((x, features), dim=1)
        x = self.dropout_comb(self.relu_comb(self.bn_comb(self.fc_comb(x))))

        x = self.dropout3(self.relu3(self.bn3(self.fc3(x))))
        x = self.fc4(x)
        return x


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
# Instantiate model, loss function, and optimizer
num_classes = 4
model = SimpleCNN(num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 100
all_metrics = []

best_f1 = -100
best_epoch = 0
best_metric = 0
with open('Results/resnet50_pretrained.csv', 'w') as f:
    f.write("Epoch,Train Loss,Val Loss,Accuracy,Precision,Recall,F1 Score,AUC,AUPR,Specificity,MCC\n")
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for images, features, labels in train_loader:
            images, features, labels = images.to(device), features.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images, features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        train_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}")

        # Validation loop
        model.eval()
        running_loss = 0.0
        with torch.no_grad():
            y_pred = []
            y_true = []
            y_proba = []
            for images, features, labels in val_loader:
                images, features, labels = images.to(device), features.to(device), labels.to(device)

                outputs = model(images, features)
                loss = criterion(outputs, labels)

                running_loss += loss.item() * images.size(0)

                # Apply softmax to get probabilities
                probabilities = F.softmax(outputs, dim=1)

                # Get the predicted class and corresponding probability
                proba, predicted = torch.max(probabilities, 1)

                # Store results
                y_pred.extend(predicted.cpu().tolist())
                y_true.extend(labels.cpu().tolist())
                y_proba.extend(probabilities.cpu().tolist())

            val_loss = running_loss / len(val_loader.dataset)
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {val_loss:.4f}")
            # Convert lists to tensors
            y_pred = torch.tensor(y_pred)
            y_true = torch.tensor(y_true)
            y_proba = torch.tensor(y_proba)

            metrics = calculate_metrics(y_pred, y_proba, y_true, num_classes)
            all_metrics.append(metrics)

            if best_f1 < float(metrics["f1_score"]):
                best_f1 = float(metrics["f1_score"])
                best_epoch = epoch+1
                best_metric = metrics
            print('---------------------------------------------------------------------------------------------------')

            f.write(f"{epoch+1},{train_loss},{val_loss},{metrics['accuracy']},{metrics['precision']},{metrics['recall']},{metrics['f1_score']},{metrics['auc']},{metrics['aupr']},{metrics['macro_specificity']},{metrics['macro_mcc']}\n")
            f.flush()
    print("Training and validating complete")

<ipython-input-14-54d133f9d5eb>:155: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image_path = os.path.join(dataset_path, row[1], row[0])


1262


<ipython-input-14-54d133f9d5eb>:179: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  image = torch.tensor(image, dtype=torch.float32)


1762


<ipython-input-14-54d133f9d5eb>:188: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image_path = os.path.join(dataset_path, row[1], row[0])


316
(1762, 512)
(316, 512)


model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

Epoch 1/100, Loss: 1.2454
Epoch 1/100, Loss: 1.0201
Accuracy: 0.6281
Precision: 0.6736
Recall: 0.6281
F1 Score: 0.6250
AUC: 0.8634
AUPR: 0.6771
Specificity per class: [0.8491379310344828, 0.9045226130653267, 0.8755555555555555, 0.9863013698630136]
Macro-average Specificity: 0.9039
MCC per class: [0.5428261550524173, 0.6052135880305155, 0.7826709811685572, 0.25797486094539523]
Macro-average MCC: 0.5472
F1 Score (micro): 0.7278
F1 Score (weighted): 0.7125
---------------------------------------------------------------------------------------------------
Epoch 2/100, Loss: 0.9460
Epoch 2/100, Loss: 0.7355
Accuracy: 0.6612
Precision: 0.8476
Recall: 0.6612
F1 Score: 0.6368
AUC: 0.9215
AUPR: 0.7454
Specificity per class: [0.8318965517241379, 0.9296482412060302, 0.9422222222222222, 1.0]
Macro-average Specificity: 0.9259
MCC per class: [0.6763988573985117, 0.6690311672687865, 0.8996259975843229, 0.1965307574511904]
Macro-average MCC: 0.6104
F1 Score (micro): 0.7911
F1 Score (weighted): 0.7635


# swinv2_tiny_window8_256 pretrained

In [ ]:
import os
import pandas as pd
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision.models as models
import numpy as np
import timm
import torch
import torch.nn.functional as F
import numpy as np
from torcheval.metrics import MulticlassAccuracy, MulticlassPrecision, MulticlassRecall, MulticlassF1Score, MulticlassAUROC, MulticlassAUPRC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
import random


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)  # For CPU
    torch.cuda.manual_seed(seed)  # For GPU (if used)
    torch.cuda.manual_seed_all(seed)  # For all GPUs
    torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior
    torch.backends.cudnn.benchmark = False


def calculate_metrics(predicted, probabilities, labels, num_classes):
    # Initialize Torcheval metrics
    accuracy_metric = MulticlassAccuracy(num_classes=num_classes, average="macro")
    precision_metric = MulticlassPrecision(num_classes=num_classes, average="macro")
    recall_metric = MulticlassRecall(num_classes=num_classes, average="macro")
    f1_score_metric = MulticlassF1Score(num_classes=num_classes, average="macro")
    auc_metric = MulticlassAUROC(num_classes=num_classes, average="macro")
    aupr_metric = MulticlassAUPRC(num_classes=num_classes, average="macro")

    # Update metrics with predictions
    accuracy_metric.update(predicted, labels)
    precision_metric.update(predicted, labels)
    recall_metric.update(predicted, labels)
    f1_score_metric.update(predicted, labels)
    auc_metric.update(probabilities, labels)
    aupr_metric.update(probabilities, labels)

    # Compute metrics
    accuracy = accuracy_metric.compute()
    precision = precision_metric.compute()
    recall = recall_metric.compute()
    f1_score = f1_score_metric.compute()
    auc = auc_metric.compute()
    aupr = aupr_metric.compute()

    # Specificity calculation
    conf_matrix = confusion_matrix(labels.cpu().numpy(), predicted.cpu().numpy(), labels=range(num_classes))
    specificity_per_class = []

    for i in range(num_classes):
        TP = conf_matrix[i, i]
        FP = conf_matrix[:, i].sum() - TP
        FN = conf_matrix[i, :].sum() - TP
        TN = conf_matrix.sum() - (TP + FP + FN)

        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
        specificity_per_class.append(specificity)

    macro_specificity = np.mean(specificity_per_class)

    # MCC calculation for multi-class classification
    mcc_per_class = []
    for i in range(num_classes):
        # Treat each class as the positive class and others as negative
        binary_pred = (predicted == i).int()
        binary_labels = (labels == i).int()
        mcc = matthews_corrcoef(binary_labels.cpu().numpy(), binary_pred.cpu().numpy())
        mcc_per_class.append(mcc)

    macro_mcc = np.mean(mcc_per_class)

    # Print all metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")
    print(f"AUC: {auc:.4f}")
    print(f"AUPR: {aupr:.4f}")
    print(f"Specificity per class: {specificity_per_class}")
    print(f"Macro-average Specificity: {macro_specificity:.4f}")
    print(f"MCC per class: {mcc_per_class}")
    print(f"Macro-average MCC: {macro_mcc:.4f}")

    f1_score_metric_micro = MulticlassF1Score(num_classes=num_classes, average="micro")
    f1_score_metric_micro.update(predicted, labels)
    f1_score_micro = f1_score_metric_micro.compute()
    print(f"F1 Score (micro): {f1_score_micro:.4f}")

    f1_score_metric_weighted = MulticlassF1Score(num_classes=num_classes, average="weighted")
    f1_score_metric_weighted.update(predicted, labels)
    f1_score_weighted = f1_score_metric_weighted.compute()
    print(f"F1 Score (weighted): {f1_score_weighted:.4f}")


    # Return metrics as a dictionary
    return {
        "accuracy": "{:.4f}".format(accuracy.item()),
        "precision": "{:.4f}".format(precision.item()),
        "recall": "{:.4f}".format(recall.item()),
        "f1_score": "{:.4f}".format(f1_score.item()),
        "auc": "{:.4f}".format(auc.item()),
        "aupr": "{:.4f}".format(aupr.item()),
        "macro_specificity": "{:.4f}".format(macro_specificity.item()),
        "macro_mcc": "{:.4f}".format(macro_mcc.item())
    }


class ImageDataset(Dataset):
    def __init__(self, images, features, labels, transform=None):
        self.images = images
        self.features = features
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        feature = self.features[idx]
        label = self.labels[idx]
        return image, feature, label


# Image transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

set_seed(42)

dataset_path = "baldder_tissue_classification"

train_df = pd.read_csv("train_df.csv")
val_df = pd.read_csv("val_df.csv")

train_images = []
train_labels = []
for idx, row in train_df.iterrows():
    image_path = os.path.join(dataset_path, row[1], row[0])
    image = Image.open(image_path)
    image = transform(image)
    train_images.append(image)
    train_labels.append(row['encoded_label'])

loaded_images = np.load("top_images.npy")
loaded_labels = np.load("top_labels.npy")
print(len(train_images))
for i in range(len(loaded_images)):
    image = loaded_images[i]
    label = loaded_labels[i]

    # Denormalize the image
    image = (image * np.array([0.229, 0.224, 0.225])[:, None, None]) + np.array([0.485, 0.456, 0.406])[:, None, None]
    image = (image * 255).astype(np.uint8)  # Convert to integer type for PIL compatibility

    # Convert NumPy array to PIL image
    image = Image.fromarray(image.transpose(1, 2, 0))  # H x W x C format

    # Apply transformations
    image = transform(image)

    # Convert back to PyTorch tensor (if required)
    image = torch.tensor(image, dtype=torch.float32)
    train_images.append(image)
    train_labels.append(label)

print(len(train_images))

val_images = []
val_labels = []
for idx, row in val_df.iterrows():
    image_path = os.path.join(dataset_path, row[1], row[0])
    image = Image.open(image_path)
    image = transform(image)
    val_images.append(image)
    val_labels.append(row['encoded_label'])

print(len(val_images))

train_features = np.load("features_RFE.npy")[:-val_df.shape[0]]
augmented_features = np.load("features_top_images_RFE.npy")
train_features = np.concatenate((train_features, augmented_features), axis=0)
val_features = np.load("features_RFE.npy")[-val_df.shape[0]:]

print(train_features.shape)
print(val_features.shape)

# Create DataLoaders
train_dataset = ImageDataset(train_images, train_features, train_labels, transform=transform)
val_dataset = ImageDataset(val_images, val_features, val_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.block = timm.create_model('swinv2_tiny_window8_256.ms_in1k', pretrained=True)
        # self.block.head = nn.Linear(self.block.num_features, 1000)

        self.fc1 = nn.Linear(1000, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)

        self.fc_ann1 = nn.Linear(512, 256)
        self.bn_ann1 = nn.BatchNorm1d(256)
        self.relu_ann1 = nn.ReLU()
        self.dropout_ann1 = nn.Dropout(0.2)
        self.fc_ann2 = nn.Linear(256, 128)
        self.bn_ann2 = nn.BatchNorm1d(128)
        self.relu_ann2 = nn.ReLU()
        self.dropout_ann2 = nn.Dropout(0.2)

        self.fc_comb = nn.Linear(384, 256)
        self.bn_comb = nn.BatchNorm1d(256)
        self.relu_comb = nn.ReLU()
        self.dropout_comb = nn.Dropout(0.2)

        self.fc3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.2)
        self.fc4 = nn.Linear(128, num_classes)

        self.relu = nn.ReLU()


    def forward(self, x, features):
        features = features.float()
        x = self.block(x)
        # print(x.shape)
        x = x.view(-1, 1000)  # Flatten
        x = self.dropout1(self.relu1(self.bn1(self.fc1(x))))
        x = self.dropout2(self.relu2(self.bn2(self.fc2(x))))

        features = self.dropout_ann1(self.relu_ann1(self.bn_ann1(self.fc_ann1(features))))
        features = self.dropout_ann2(self.relu_ann2(self.bn_ann2(self.fc_ann2(features))))

        x = torch.cat((x, features), dim=1)
        x = self.dropout_comb(self.relu_comb(self.bn_comb(self.fc_comb(x))))

        x = self.dropout3(self.relu3(self.bn3(self.fc3(x))))
        x = self.fc4(x)
        return x


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
# Instantiate model, loss function, and optimizer
num_classes = 4
model = SimpleCNN(num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 100
all_metrics = []

best_f1 = -100
best_epoch = 0
best_metric = 0
with open('Results/swinv2_tiny_window8_256_pretrained.csv', 'w') as f:
    f.write("Epoch,Train Loss,Val Loss,Accuracy,Precision,Recall,F1 Score,AUC,AUPR,Specificity,MCC\n")
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for images, features, labels in train_loader:
            images, features, labels = images.to(device), features.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images, features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        train_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}")

        # Validation loop
        model.eval()
        running_loss = 0.0
        with torch.no_grad():
            y_pred = []
            y_true = []
            y_proba = []
            for images, features, labels in val_loader:
                images, features, labels = images.to(device), features.to(device), labels.to(device)

                outputs = model(images, features)
                loss = criterion(outputs, labels)

                running_loss += loss.item() * images.size(0)

                # Apply softmax to get probabilities
                probabilities = F.softmax(outputs, dim=1)

                # Get the predicted class and corresponding probability
                proba, predicted = torch.max(probabilities, 1)

                # Store results
                y_pred.extend(predicted.cpu().tolist())
                y_true.extend(labels.cpu().tolist())
                y_proba.extend(probabilities.cpu().tolist())

            val_loss = running_loss / len(val_loader.dataset)
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {val_loss:.4f}")
            # Convert lists to tensors
            y_pred = torch.tensor(y_pred)
            y_true = torch.tensor(y_true)
            y_proba = torch.tensor(y_proba)

            metrics = calculate_metrics(y_pred, y_proba, y_true, num_classes)
            all_metrics.append(metrics)

            if best_f1 < float(metrics["f1_score"]):
                best_f1 = float(metrics["f1_score"])
                best_epoch = epoch+1
                best_metric = metrics
            print('---------------------------------------------------------------------------------------------------')

            f.write(f"{epoch+1},{train_loss},{val_loss},{metrics['accuracy']},{metrics['precision']},{metrics['recall']},{metrics['f1_score']},{metrics['auc']},{metrics['aupr']},{metrics['macro_specificity']},{metrics['macro_mcc']}\n")
            f.flush()
    print("Training and validating complete")

<ipython-input-17-bd95416032f4>:155: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image_path = os.path.join(dataset_path, row[1], row[0])


1262


<ipython-input-17-bd95416032f4>:179: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  image = torch.tensor(image, dtype=torch.float32)


1762


<ipython-input-17-bd95416032f4>:188: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image_path = os.path.join(dataset_path, row[1], row[0])


316
(1762, 512)
(316, 512)


model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

Epoch 1/100, Loss: 1.0639
Epoch 1/100, Loss: 0.6123
Accuracy: 0.7697
Precision: 0.8716
Recall: 0.7697
F1 Score: 0.7931
AUC: 0.9622
AUPR: 0.8818
Specificity per class: [1.0, 0.9095477386934674, 0.8844444444444445, 0.9931506849315068]
Macro-average Specificity: 0.9468
MCC per class: [0.7628961803617725, 0.8579945078563536, 0.829398115845401, 0.6022078852755764]
Macro-average MCC: 0.7631
F1 Score (micro): 0.8544
F1 Score (weighted): 0.8449
---------------------------------------------------------------------------------------------------
Epoch 2/100, Loss: 0.7498
Epoch 2/100, Loss: 0.6010
Accuracy: 0.7392
Precision: 0.8515
Recall: 0.7392
F1 Score: 0.7378
AUC: 0.9600
AUPR: 0.8622
Specificity per class: [0.875, 0.9899497487437185, 0.9511111111111111, 0.9965753424657534]
Macro-average Specificity: 0.9532
MCC per class: [0.7689369528217611, 0.8591921352460316, 0.9211624748169529, 0.3465483067411199]
Macro-average MCC: 0.7240
F1 Score (micro): 0.8639
F1 Score (weighted): 0.8473
---------------

# densenet201 pretrained

In [ ]:
import os
import pandas as pd
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision.models as models
import numpy as np
import timm
import torch
import torch.nn.functional as F
import numpy as np
from torcheval.metrics import MulticlassAccuracy, MulticlassPrecision, MulticlassRecall, MulticlassF1Score, MulticlassAUROC, MulticlassAUPRC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
import random


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)  # For CPU
    torch.cuda.manual_seed(seed)  # For GPU (if used)
    torch.cuda.manual_seed_all(seed)  # For all GPUs
    torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior
    torch.backends.cudnn.benchmark = False


def calculate_metrics(predicted, probabilities, labels, num_classes):
    # Initialize Torcheval metrics
    accuracy_metric = MulticlassAccuracy(num_classes=num_classes, average="macro")
    precision_metric = MulticlassPrecision(num_classes=num_classes, average="macro")
    recall_metric = MulticlassRecall(num_classes=num_classes, average="macro")
    f1_score_metric = MulticlassF1Score(num_classes=num_classes, average="macro")
    auc_metric = MulticlassAUROC(num_classes=num_classes, average="macro")
    aupr_metric = MulticlassAUPRC(num_classes=num_classes, average="macro")

    # Update metrics with predictions
    accuracy_metric.update(predicted, labels)
    precision_metric.update(predicted, labels)
    recall_metric.update(predicted, labels)
    f1_score_metric.update(predicted, labels)
    auc_metric.update(probabilities, labels)
    aupr_metric.update(probabilities, labels)

    # Compute metrics
    accuracy = accuracy_metric.compute()
    precision = precision_metric.compute()
    recall = recall_metric.compute()
    f1_score = f1_score_metric.compute()
    auc = auc_metric.compute()
    aupr = aupr_metric.compute()

    # Specificity calculation
    conf_matrix = confusion_matrix(labels.cpu().numpy(), predicted.cpu().numpy(), labels=range(num_classes))
    specificity_per_class = []

    for i in range(num_classes):
        TP = conf_matrix[i, i]
        FP = conf_matrix[:, i].sum() - TP
        FN = conf_matrix[i, :].sum() - TP
        TN = conf_matrix.sum() - (TP + FP + FN)

        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
        specificity_per_class.append(specificity)

    macro_specificity = np.mean(specificity_per_class)

    # MCC calculation for multi-class classification
    mcc_per_class = []
    for i in range(num_classes):
        # Treat each class as the positive class and others as negative
        binary_pred = (predicted == i).int()
        binary_labels = (labels == i).int()
        mcc = matthews_corrcoef(binary_labels.cpu().numpy(), binary_pred.cpu().numpy())
        mcc_per_class.append(mcc)

    macro_mcc = np.mean(mcc_per_class)

    # Print all metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")
    print(f"AUC: {auc:.4f}")
    print(f"AUPR: {aupr:.4f}")
    print(f"Specificity per class: {specificity_per_class}")
    print(f"Macro-average Specificity: {macro_specificity:.4f}")
    print(f"MCC per class: {mcc_per_class}")
    print(f"Macro-average MCC: {macro_mcc:.4f}")

    f1_score_metric_micro = MulticlassF1Score(num_classes=num_classes, average="micro")
    f1_score_metric_micro.update(predicted, labels)
    f1_score_micro = f1_score_metric_micro.compute()
    print(f"F1 Score (micro): {f1_score_micro:.4f}")

    f1_score_metric_weighted = MulticlassF1Score(num_classes=num_classes, average="weighted")
    f1_score_metric_weighted.update(predicted, labels)
    f1_score_weighted = f1_score_metric_weighted.compute()
    print(f"F1 Score (weighted): {f1_score_weighted:.4f}")


    # Return metrics as a dictionary
    return {
        "accuracy": "{:.4f}".format(accuracy.item()),
        "precision": "{:.4f}".format(precision.item()),
        "recall": "{:.4f}".format(recall.item()),
        "f1_score": "{:.4f}".format(f1_score.item()),
        "auc": "{:.4f}".format(auc.item()),
        "aupr": "{:.4f}".format(aupr.item()),
        "macro_specificity": "{:.4f}".format(macro_specificity.item()),
        "macro_mcc": "{:.4f}".format(macro_mcc.item())
    }


class ImageDataset(Dataset):
    def __init__(self, images, features, labels, transform=None):
        self.images = images
        self.features = features
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        feature = self.features[idx]
        label = self.labels[idx]
        return image, feature, label


# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

set_seed(42)

dataset_path = "baldder_tissue_classification"

train_df = pd.read_csv("train_df.csv")
val_df = pd.read_csv("val_df.csv")

train_images = []
train_labels = []
for idx, row in train_df.iterrows():
    image_path = os.path.join(dataset_path, row[1], row[0])
    image = Image.open(image_path)
    image = transform(image)
    train_images.append(image)
    train_labels.append(row['encoded_label'])

loaded_images = np.load("top_images.npy")
loaded_labels = np.load("top_labels.npy")
print(len(train_images))
for i in range(len(loaded_images)):
    image = loaded_images[i]
    label = loaded_labels[i]

    # Denormalize the image
    image = (image * np.array([0.229, 0.224, 0.225])[:, None, None]) + np.array([0.485, 0.456, 0.406])[:, None, None]
    image = (image * 255).astype(np.uint8)  # Convert to integer type for PIL compatibility

    # Convert NumPy array to PIL image
    image = Image.fromarray(image.transpose(1, 2, 0))  # H x W x C format

    # Apply transformations
    image = transform(image)

    # Convert back to PyTorch tensor (if required)
    image = torch.tensor(image, dtype=torch.float32)
    train_images.append(image)
    train_labels.append(label)

print(len(train_images))

val_images = []
val_labels = []
for idx, row in val_df.iterrows():
    image_path = os.path.join(dataset_path, row[1], row[0])
    image = Image.open(image_path)
    image = transform(image)
    val_images.append(image)
    val_labels.append(row['encoded_label'])

print(len(val_images))

train_features = np.load("features_RFE.npy")[:-val_df.shape[0]]
augmented_features = np.load("features_top_images_RFE.npy")
train_features = np.concatenate((train_features, augmented_features), axis=0)
val_features = np.load("features_RFE.npy")[-val_df.shape[0]:]

print(train_features.shape)
print(val_features.shape)

# Create DataLoaders
train_dataset = ImageDataset(train_images, train_features, train_labels, transform=transform)
val_dataset = ImageDataset(val_images, val_features, val_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.block = timm.create_model('densenet201.tv_in1k', pretrained=True)
        # self.block.head = nn.Linear(self.block.num_features, 1000)

        self.fc1 = nn.Linear(1000, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)

        self.fc_ann1 = nn.Linear(512, 256)
        self.bn_ann1 = nn.BatchNorm1d(256)
        self.relu_ann1 = nn.ReLU()
        self.dropout_ann1 = nn.Dropout(0.2)
        self.fc_ann2 = nn.Linear(256, 128)
        self.bn_ann2 = nn.BatchNorm1d(128)
        self.relu_ann2 = nn.ReLU()
        self.dropout_ann2 = nn.Dropout(0.2)

        self.fc_comb = nn.Linear(384, 256)
        self.bn_comb = nn.BatchNorm1d(256)
        self.relu_comb = nn.ReLU()
        self.dropout_comb = nn.Dropout(0.2)

        self.fc3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.2)
        self.fc4 = nn.Linear(128, num_classes)

        self.relu = nn.ReLU()


    def forward(self, x, features):
        features = features.float()
        x = self.block(x)
        # print(x.shape)
        x = x.view(-1, 1000)  # Flatten
        x = self.dropout1(self.relu1(self.bn1(self.fc1(x))))
        x = self.dropout2(self.relu2(self.bn2(self.fc2(x))))

        features = self.dropout_ann1(self.relu_ann1(self.bn_ann1(self.fc_ann1(features))))
        features = self.dropout_ann2(self.relu_ann2(self.bn_ann2(self.fc_ann2(features))))

        x = torch.cat((x, features), dim=1)
        x = self.dropout_comb(self.relu_comb(self.bn_comb(self.fc_comb(x))))

        x = self.dropout3(self.relu3(self.bn3(self.fc3(x))))
        x = self.fc4(x)
        return x


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
# Instantiate model, loss function, and optimizer
num_classes = 4
model = SimpleCNN(num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 100
all_metrics = []

best_f1 = -100
best_epoch = 0
best_metric = 0
with open('Results/densenet201_pretrained.csv', 'w') as f:
    f.write("Epoch,Train Loss,Val Loss,Accuracy,Precision,Recall,F1 Score,AUC,AUPR,Specificity,MCC\n")
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for images, features, labels in train_loader:
            images, features, labels = images.to(device), features.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images, features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        train_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}")

        # Validation loop
        model.eval()
        running_loss = 0.0
        with torch.no_grad():
            y_pred = []
            y_true = []
            y_proba = []
            for images, features, labels in val_loader:
                images, features, labels = images.to(device), features.to(device), labels.to(device)

                outputs = model(images, features)
                loss = criterion(outputs, labels)

                running_loss += loss.item() * images.size(0)

                # Apply softmax to get probabilities
                probabilities = F.softmax(outputs, dim=1)

                # Get the predicted class and corresponding probability
                proba, predicted = torch.max(probabilities, 1)

                # Store results
                y_pred.extend(predicted.cpu().tolist())
                y_true.extend(labels.cpu().tolist())
                y_proba.extend(probabilities.cpu().tolist())

            val_loss = running_loss / len(val_loader.dataset)
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {val_loss:.4f}")
            # Convert lists to tensors
            y_pred = torch.tensor(y_pred)
            y_true = torch.tensor(y_true)
            y_proba = torch.tensor(y_proba)

            metrics = calculate_metrics(y_pred, y_proba, y_true, num_classes)
            all_metrics.append(metrics)

            if best_f1 < float(metrics["f1_score"]):
                best_f1 = float(metrics["f1_score"])
                best_epoch = epoch+1
                best_metric = metrics
            print('---------------------------------------------------------------------------------------------------')

            f.write(f"{epoch+1},{train_loss},{val_loss},{metrics['accuracy']},{metrics['precision']},{metrics['recall']},{metrics['f1_score']},{metrics['auc']},{metrics['aupr']},{metrics['macro_specificity']},{metrics['macro_mcc']}\n")
            f.flush()
    print("Training and validating complete")

<ipython-input-19-1e6270576aaf>:155: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image_path = os.path.join(dataset_path, row[1], row[0])


1262


<ipython-input-19-1e6270576aaf>:179: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  image = torch.tensor(image, dtype=torch.float32)


1762


<ipython-input-19-1e6270576aaf>:188: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image_path = os.path.join(dataset_path, row[1], row[0])


316
(1762, 512)
(316, 512)


model.safetensors:   0%|          | 0.00/81.1M [00:00<?, ?B/s]

Epoch 1/100, Loss: 1.0940
Epoch 1/100, Loss: 0.5481
Accuracy: 0.7882
Precision: 0.9360
Recall: 0.7882
F1 Score: 0.8051
AUC: 0.9844
AUPR: 0.9283
Specificity per class: [0.9698275862068966, 0.9346733668341709, 0.9688888888888889, 1.0]
Macro-average Specificity: 0.9683
MCC per class: [0.8950297057731631, 0.89529475693873, 0.9485159670150733, 0.4852668039004337]
Macro-average MCC: 0.8060
F1 Score (micro): 0.9146
F1 Score (weighted): 0.8990
---------------------------------------------------------------------------------------------------
Epoch 2/100, Loss: 0.5833
Epoch 2/100, Loss: 0.3756
Accuracy: 0.9020
Precision: 0.9507
Recall: 0.9020
F1 Score: 0.9199
AUC: 0.9874
AUPR: 0.9604
Specificity per class: [0.9612068965517241, 0.9949748743718593, 0.9866666666666667, 0.9965753424657534]
Macro-average Specificity: 0.9849
MCC per class: [0.9317654609958362, 0.9728500343618371, 0.9536996336996337, 0.7787696499643428]
Macro-average MCC: 0.9093
F1 Score (micro): 0.9557
F1 Score (weighted): 0.9539
---

# efficientformerv2 pretrained

In [ ]:
import os
import pandas as pd
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision.models as models
import numpy as np
import timm
import torch
import torch.nn.functional as F
import numpy as np
from torcheval.metrics import MulticlassAccuracy, MulticlassPrecision, MulticlassRecall, MulticlassF1Score, MulticlassAUROC, MulticlassAUPRC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
import random


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)  # For CPU
    torch.cuda.manual_seed(seed)  # For GPU (if used)
    torch.cuda.manual_seed_all(seed)  # For all GPUs
    torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior
    torch.backends.cudnn.benchmark = False


def calculate_metrics(predicted, probabilities, labels, num_classes):
    # Initialize Torcheval metrics
    accuracy_metric = MulticlassAccuracy(num_classes=num_classes, average="macro")
    precision_metric = MulticlassPrecision(num_classes=num_classes, average="macro")
    recall_metric = MulticlassRecall(num_classes=num_classes, average="macro")
    f1_score_metric = MulticlassF1Score(num_classes=num_classes, average="macro")
    auc_metric = MulticlassAUROC(num_classes=num_classes, average="macro")
    aupr_metric = MulticlassAUPRC(num_classes=num_classes, average="macro")

    # Update metrics with predictions
    accuracy_metric.update(predicted, labels)
    precision_metric.update(predicted, labels)
    recall_metric.update(predicted, labels)
    f1_score_metric.update(predicted, labels)
    auc_metric.update(probabilities, labels)
    aupr_metric.update(probabilities, labels)

    # Compute metrics
    accuracy = accuracy_metric.compute()
    precision = precision_metric.compute()
    recall = recall_metric.compute()
    f1_score = f1_score_metric.compute()
    auc = auc_metric.compute()
    aupr = aupr_metric.compute()

    # Specificity calculation
    conf_matrix = confusion_matrix(labels.cpu().numpy(), predicted.cpu().numpy(), labels=range(num_classes))
    specificity_per_class = []

    for i in range(num_classes):
        TP = conf_matrix[i, i]
        FP = conf_matrix[:, i].sum() - TP
        FN = conf_matrix[i, :].sum() - TP
        TN = conf_matrix.sum() - (TP + FP + FN)

        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
        specificity_per_class.append(specificity)

    macro_specificity = np.mean(specificity_per_class)

    # MCC calculation for multi-class classification
    mcc_per_class = []
    for i in range(num_classes):
        # Treat each class as the positive class and others as negative
        binary_pred = (predicted == i).int()
        binary_labels = (labels == i).int()
        mcc = matthews_corrcoef(binary_labels.cpu().numpy(), binary_pred.cpu().numpy())
        mcc_per_class.append(mcc)

    macro_mcc = np.mean(mcc_per_class)

    # Print all metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")
    print(f"AUC: {auc:.4f}")
    print(f"AUPR: {aupr:.4f}")
    print(f"Specificity per class: {specificity_per_class}")
    print(f"Macro-average Specificity: {macro_specificity:.4f}")
    print(f"MCC per class: {mcc_per_class}")
    print(f"Macro-average MCC: {macro_mcc:.4f}")

    f1_score_metric_micro = MulticlassF1Score(num_classes=num_classes, average="micro")
    f1_score_metric_micro.update(predicted, labels)
    f1_score_micro = f1_score_metric_micro.compute()
    print(f"F1 Score (micro): {f1_score_micro:.4f}")

    f1_score_metric_weighted = MulticlassF1Score(num_classes=num_classes, average="weighted")
    f1_score_metric_weighted.update(predicted, labels)
    f1_score_weighted = f1_score_metric_weighted.compute()
    print(f"F1 Score (weighted): {f1_score_weighted:.4f}")


    # Return metrics as a dictionary
    return {
        "accuracy": "{:.4f}".format(accuracy.item()),
        "precision": "{:.4f}".format(precision.item()),
        "recall": "{:.4f}".format(recall.item()),
        "f1_score": "{:.4f}".format(f1_score.item()),
        "auc": "{:.4f}".format(auc.item()),
        "aupr": "{:.4f}".format(aupr.item()),
        "macro_specificity": "{:.4f}".format(macro_specificity.item()),
        "macro_mcc": "{:.4f}".format(macro_mcc.item())
    }


class ImageDataset(Dataset):
    def __init__(self, images, features, labels, transform=None):
        self.images = images
        self.features = features
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        feature = self.features[idx]
        label = self.labels[idx]
        return image, feature, label


# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

set_seed(42)

dataset_path = "baldder_tissue_classification"

train_df = pd.read_csv("train_df.csv")
val_df = pd.read_csv("val_df.csv")

train_images = []
train_labels = []
for idx, row in train_df.iterrows():
    image_path = os.path.join(dataset_path, row[1], row[0])
    image = Image.open(image_path)
    image = transform(image)
    train_images.append(image)
    train_labels.append(row['encoded_label'])

loaded_images = np.load("top_images.npy")
loaded_labels = np.load("top_labels.npy")
print(len(train_images))
for i in range(len(loaded_images)):
    image = loaded_images[i]
    label = loaded_labels[i]

    # Denormalize the image
    image = (image * np.array([0.229, 0.224, 0.225])[:, None, None]) + np.array([0.485, 0.456, 0.406])[:, None, None]
    image = (image * 255).astype(np.uint8)  # Convert to integer type for PIL compatibility

    # Convert NumPy array to PIL image
    image = Image.fromarray(image.transpose(1, 2, 0))  # H x W x C format

    # Apply transformations
    image = transform(image)

    # Convert back to PyTorch tensor (if required)
    image = torch.tensor(image, dtype=torch.float32)
    train_images.append(image)
    train_labels.append(label)

print(len(train_images))

val_images = []
val_labels = []
for idx, row in val_df.iterrows():
    image_path = os.path.join(dataset_path, row[1], row[0])
    image = Image.open(image_path)
    image = transform(image)
    val_images.append(image)
    val_labels.append(row['encoded_label'])

print(len(val_images))

train_features = np.load("features_RFE.npy")[:-val_df.shape[0]]
augmented_features = np.load("features_top_images_RFE.npy")
train_features = np.concatenate((train_features, augmented_features), axis=0)
val_features = np.load("features_RFE.npy")[-val_df.shape[0]:]

print(train_features.shape)
print(val_features.shape)

# Create DataLoaders
train_dataset = ImageDataset(train_images, train_features, train_labels, transform=transform)
val_dataset = ImageDataset(val_images, val_features, val_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.block = timm.create_model('efficientformerv2_l.snap_dist_in1k', pretrained=True)
        # self.block.head = nn.Linear(self.block.num_features, 1000)

        self.fc1 = nn.Linear(1000, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)

        self.fc_ann1 = nn.Linear(512, 256)
        self.bn_ann1 = nn.BatchNorm1d(256)
        self.relu_ann1 = nn.ReLU()
        self.dropout_ann1 = nn.Dropout(0.2)
        self.fc_ann2 = nn.Linear(256, 128)
        self.bn_ann2 = nn.BatchNorm1d(128)
        self.relu_ann2 = nn.ReLU()
        self.dropout_ann2 = nn.Dropout(0.2)

        self.fc_comb = nn.Linear(384, 256)
        self.bn_comb = nn.BatchNorm1d(256)
        self.relu_comb = nn.ReLU()
        self.dropout_comb = nn.Dropout(0.2)

        self.fc3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.2)
        self.fc4 = nn.Linear(128, num_classes)

        self.relu = nn.ReLU()


    def forward(self, x, features):
        features = features.float()
        x = self.block(x)
        # print(x.shape)
        x = x.view(-1, 1000)  # Flatten
        x = self.dropout1(self.relu1(self.bn1(self.fc1(x))))
        x = self.dropout2(self.relu2(self.bn2(self.fc2(x))))

        features = self.dropout_ann1(self.relu_ann1(self.bn_ann1(self.fc_ann1(features))))
        features = self.dropout_ann2(self.relu_ann2(self.bn_ann2(self.fc_ann2(features))))

        x = torch.cat((x, features), dim=1)
        x = self.dropout_comb(self.relu_comb(self.bn_comb(self.fc_comb(x))))

        x = self.dropout3(self.relu3(self.bn3(self.fc3(x))))
        x = self.fc4(x)
        return x


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
# Instantiate model, loss function, and optimizer
num_classes = 4
model = SimpleCNN(num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 100
all_metrics = []

best_f1 = -100
best_epoch = 0
best_metric = 0
with open('Results/efficientformerv2_pretrained.csv', 'w') as f:
    f.write("Epoch,Train Loss,Val Loss,Accuracy,Precision,Recall,F1 Score,AUC,AUPR,Specificity,MCC\n")
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for images, features, labels in train_loader:
            images, features, labels = images.to(device), features.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images, features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        train_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}")

        # Validation loop
        model.eval()
        running_loss = 0.0
        with torch.no_grad():
            y_pred = []
            y_true = []
            y_proba = []
            for images, features, labels in val_loader:
                images, features, labels = images.to(device), features.to(device), labels.to(device)

                outputs = model(images, features)
                loss = criterion(outputs, labels)

                running_loss += loss.item() * images.size(0)

                # Apply softmax to get probabilities
                probabilities = F.softmax(outputs, dim=1)

                # Get the predicted class and corresponding probability
                proba, predicted = torch.max(probabilities, 1)

                # Store results
                y_pred.extend(predicted.cpu().tolist())
                y_true.extend(labels.cpu().tolist())
                y_proba.extend(probabilities.cpu().tolist())

            val_loss = running_loss / len(val_loader.dataset)
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {val_loss:.4f}")
            # Convert lists to tensors
            y_pred = torch.tensor(y_pred)
            y_true = torch.tensor(y_true)
            y_proba = torch.tensor(y_proba)

            metrics = calculate_metrics(y_pred, y_proba, y_true, num_classes)
            all_metrics.append(metrics)

            if best_f1 < float(metrics["f1_score"]):
                best_f1 = float(metrics["f1_score"])
                best_epoch = epoch+1
                best_metric = metrics
            print('---------------------------------------------------------------------------------------------------')

            f.write(f"{epoch+1},{train_loss},{val_loss},{metrics['accuracy']},{metrics['precision']},{metrics['recall']},{metrics['f1_score']},{metrics['auc']},{metrics['aupr']},{metrics['macro_specificity']},{metrics['macro_mcc']}\n")
            f.flush()
    print("Training and validating complete")

<ipython-input-21-0c9ff24b5fb9>:155: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image_path = os.path.join(dataset_path, row[1], row[0])


1262


<ipython-input-21-0c9ff24b5fb9>:179: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  image = torch.tensor(image, dtype=torch.float32)


1762


<ipython-input-21-0c9ff24b5fb9>:188: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image_path = os.path.join(dataset_path, row[1], row[0])


316
(1762, 512)
(316, 512)


pytorch_model.bin:   0%|          | 0.00/107M [00:00<?, ?B/s]

Epoch 1/100, Loss: 1.2642
Epoch 1/100, Loss: 0.7910
Accuracy: 0.7279
Precision: 0.8139
Recall: 0.7279
F1 Score: 0.7327
AUC: 0.9559
AUPR: 0.8151
Specificity per class: [0.9267241379310345, 0.964824120603015, 0.8977777777777778, 0.9931506849315068]
Macro-average Specificity: 0.9456
MCC per class: [0.7698712156569839, 0.8294036537526095, 0.8465501600550782, 0.36266812785466307]
Macro-average MCC: 0.7021
F1 Score (micro): 0.8449
F1 Score (weighted): 0.8302
---------------------------------------------------------------------------------------------------
Epoch 2/100, Loss: 0.8451
Epoch 2/100, Loss: 0.5205
Accuracy: 0.7144
Precision: 0.7763
Recall: 0.7144
F1 Score: 0.6954
AUC: 0.9566
AUPR: 0.8491
Specificity per class: [0.9181034482758621, 0.949748743718593, 0.9511111111111111, 0.9965753424657534]
Macro-average Specificity: 0.9539
MCC per class: [0.7836875934443235, 0.8713113657029775, 0.9211624748169529, 0.12774912247126896]
Macro-average MCC: 0.6760
F1 Score (micro): 0.8703
F1 Score (weig

# twins_pcpvt_base pretrained

In [ ]:
import os
import pandas as pd
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision.models as models
import numpy as np
import timm
import torch
import torch.nn.functional as F
import numpy as np
from torcheval.metrics import MulticlassAccuracy, MulticlassPrecision, MulticlassRecall, MulticlassF1Score, MulticlassAUROC, MulticlassAUPRC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
import random


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)  # For CPU
    torch.cuda.manual_seed(seed)  # For GPU (if used)
    torch.cuda.manual_seed_all(seed)  # For all GPUs
    torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior
    torch.backends.cudnn.benchmark = False


def calculate_metrics(predicted, probabilities, labels, num_classes):
    # Initialize Torcheval metrics
    accuracy_metric = MulticlassAccuracy(num_classes=num_classes, average="macro")
    precision_metric = MulticlassPrecision(num_classes=num_classes, average="macro")
    recall_metric = MulticlassRecall(num_classes=num_classes, average="macro")
    f1_score_metric = MulticlassF1Score(num_classes=num_classes, average="macro")
    auc_metric = MulticlassAUROC(num_classes=num_classes, average="macro")
    aupr_metric = MulticlassAUPRC(num_classes=num_classes, average="macro")

    # Update metrics with predictions
    accuracy_metric.update(predicted, labels)
    precision_metric.update(predicted, labels)
    recall_metric.update(predicted, labels)
    f1_score_metric.update(predicted, labels)
    auc_metric.update(probabilities, labels)
    aupr_metric.update(probabilities, labels)

    # Compute metrics
    accuracy = accuracy_metric.compute()
    precision = precision_metric.compute()
    recall = recall_metric.compute()
    f1_score = f1_score_metric.compute()
    auc = auc_metric.compute()
    aupr = aupr_metric.compute()

    # Specificity calculation
    conf_matrix = confusion_matrix(labels.cpu().numpy(), predicted.cpu().numpy(), labels=range(num_classes))
    specificity_per_class = []

    for i in range(num_classes):
        TP = conf_matrix[i, i]
        FP = conf_matrix[:, i].sum() - TP
        FN = conf_matrix[i, :].sum() - TP
        TN = conf_matrix.sum() - (TP + FP + FN)

        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
        specificity_per_class.append(specificity)

    macro_specificity = np.mean(specificity_per_class)

    # MCC calculation for multi-class classification
    mcc_per_class = []
    for i in range(num_classes):
        # Treat each class as the positive class and others as negative
        binary_pred = (predicted == i).int()
        binary_labels = (labels == i).int()
        mcc = matthews_corrcoef(binary_labels.cpu().numpy(), binary_pred.cpu().numpy())
        mcc_per_class.append(mcc)

    macro_mcc = np.mean(mcc_per_class)

    # Print all metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")
    print(f"AUC: {auc:.4f}")
    print(f"AUPR: {aupr:.4f}")
    print(f"Specificity per class: {specificity_per_class}")
    print(f"Macro-average Specificity: {macro_specificity:.4f}")
    print(f"MCC per class: {mcc_per_class}")
    print(f"Macro-average MCC: {macro_mcc:.4f}")

    f1_score_metric_micro = MulticlassF1Score(num_classes=num_classes, average="micro")
    f1_score_metric_micro.update(predicted, labels)
    f1_score_micro = f1_score_metric_micro.compute()
    print(f"F1 Score (micro): {f1_score_micro:.4f}")

    f1_score_metric_weighted = MulticlassF1Score(num_classes=num_classes, average="weighted")
    f1_score_metric_weighted.update(predicted, labels)
    f1_score_weighted = f1_score_metric_weighted.compute()
    print(f"F1 Score (weighted): {f1_score_weighted:.4f}")


    # Return metrics as a dictionary
    return {
        "accuracy": "{:.4f}".format(accuracy.item()),
        "precision": "{:.4f}".format(precision.item()),
        "recall": "{:.4f}".format(recall.item()),
        "f1_score": "{:.4f}".format(f1_score.item()),
        "auc": "{:.4f}".format(auc.item()),
        "aupr": "{:.4f}".format(aupr.item()),
        "macro_specificity": "{:.4f}".format(macro_specificity.item()),
        "macro_mcc": "{:.4f}".format(macro_mcc.item())
    }


class ImageDataset(Dataset):
    def __init__(self, images, features, labels, transform=None):
        self.images = images
        self.features = features
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        feature = self.features[idx]
        label = self.labels[idx]
        return image, feature, label


# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

set_seed(42)

dataset_path = "baldder_tissue_classification"

train_df = pd.read_csv("train_df.csv")
val_df = pd.read_csv("val_df.csv")

train_images = []
train_labels = []
for idx, row in train_df.iterrows():
    image_path = os.path.join(dataset_path, row[1], row[0])
    image = Image.open(image_path)
    image = transform(image)
    train_images.append(image)
    train_labels.append(row['encoded_label'])

loaded_images = np.load("top_images.npy")
loaded_labels = np.load("top_labels.npy")
print(len(train_images))
for i in range(len(loaded_images)):
    image = loaded_images[i]
    label = loaded_labels[i]

    # Denormalize the image
    image = (image * np.array([0.229, 0.224, 0.225])[:, None, None]) + np.array([0.485, 0.456, 0.406])[:, None, None]
    image = (image * 255).astype(np.uint8)  # Convert to integer type for PIL compatibility

    # Convert NumPy array to PIL image
    image = Image.fromarray(image.transpose(1, 2, 0))  # H x W x C format

    # Apply transformations
    image = transform(image)

    # Convert back to PyTorch tensor (if required)
    image = torch.tensor(image, dtype=torch.float32)
    train_images.append(image)
    train_labels.append(label)

print(len(train_images))

val_images = []
val_labels = []
for idx, row in val_df.iterrows():
    image_path = os.path.join(dataset_path, row[1], row[0])
    image = Image.open(image_path)
    image = transform(image)
    val_images.append(image)
    val_labels.append(row['encoded_label'])

print(len(val_images))

train_features = np.load("features_RFE.npy")[:-val_df.shape[0]]
augmented_features = np.load("features_top_images_RFE.npy")
train_features = np.concatenate((train_features, augmented_features), axis=0)
val_features = np.load("features_RFE.npy")[-val_df.shape[0]:]

print(train_features.shape)
print(val_features.shape)

# Create DataLoaders
train_dataset = ImageDataset(train_images, train_features, train_labels, transform=transform)
val_dataset = ImageDataset(val_images, val_features, val_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.block = timm.create_model('twins_pcpvt_base.in1k', pretrained=True)
        # self.block.head = nn.Linear(self.block.num_features, 1000)

        self.fc1 = nn.Linear(1000, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)

        self.fc_ann1 = nn.Linear(512, 256)
        self.bn_ann1 = nn.BatchNorm1d(256)
        self.relu_ann1 = nn.ReLU()
        self.dropout_ann1 = nn.Dropout(0.2)
        self.fc_ann2 = nn.Linear(256, 128)
        self.bn_ann2 = nn.BatchNorm1d(128)
        self.relu_ann2 = nn.ReLU()
        self.dropout_ann2 = nn.Dropout(0.2)

        self.fc_comb = nn.Linear(384, 256)
        self.bn_comb = nn.BatchNorm1d(256)
        self.relu_comb = nn.ReLU()
        self.dropout_comb = nn.Dropout(0.2)

        self.fc3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.2)
        self.fc4 = nn.Linear(128, num_classes)

        self.relu = nn.ReLU()


    def forward(self, x, features):
        features = features.float()
        x = self.block(x)
        # print(x.shape)
        x = x.view(-1, 1000)  # Flatten
        x = self.dropout1(self.relu1(self.bn1(self.fc1(x))))
        x = self.dropout2(self.relu2(self.bn2(self.fc2(x))))

        features = self.dropout_ann1(self.relu_ann1(self.bn_ann1(self.fc_ann1(features))))
        features = self.dropout_ann2(self.relu_ann2(self.bn_ann2(self.fc_ann2(features))))

        x = torch.cat((x, features), dim=1)
        x = self.dropout_comb(self.relu_comb(self.bn_comb(self.fc_comb(x))))

        x = self.dropout3(self.relu3(self.bn3(self.fc3(x))))
        x = self.fc4(x)
        return x


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
# Instantiate model, loss function, and optimizer
num_classes = 4
model = SimpleCNN(num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 100
all_metrics = []

best_f1 = -100
best_epoch = 0
best_metric = 0
with open('Results/twins_pcpvt_base_pretrained.csv', 'w') as f:
    f.write("Epoch,Train Loss,Val Loss,Accuracy,Precision,Recall,F1 Score,AUC,AUPR,Specificity,MCC\n")
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for images, features, labels in train_loader:
            images, features, labels = images.to(device), features.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images, features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        train_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}")

        # Validation loop
        model.eval()
        running_loss = 0.0
        with torch.no_grad():
            y_pred = []
            y_true = []
            y_proba = []
            for images, features, labels in val_loader:
                images, features, labels = images.to(device), features.to(device), labels.to(device)

                outputs = model(images, features)
                loss = criterion(outputs, labels)

                running_loss += loss.item() * images.size(0)

                # Apply softmax to get probabilities
                probabilities = F.softmax(outputs, dim=1)

                # Get the predicted class and corresponding probability
                proba, predicted = torch.max(probabilities, 1)

                # Store results
                y_pred.extend(predicted.cpu().tolist())
                y_true.extend(labels.cpu().tolist())
                y_proba.extend(probabilities.cpu().tolist())

            val_loss = running_loss / len(val_loader.dataset)
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {val_loss:.4f}")
            # Convert lists to tensors
            y_pred = torch.tensor(y_pred)
            y_true = torch.tensor(y_true)
            y_proba = torch.tensor(y_proba)

            metrics = calculate_metrics(y_pred, y_proba, y_true, num_classes)
            all_metrics.append(metrics)

            if best_f1 < float(metrics["f1_score"]):
                best_f1 = float(metrics["f1_score"])
                best_epoch = epoch+1
                best_metric = metrics
            print('---------------------------------------------------------------------------------------------------')

            f.write(f"{epoch+1},{train_loss},{val_loss},{metrics['accuracy']},{metrics['precision']},{metrics['recall']},{metrics['f1_score']},{metrics['auc']},{metrics['aupr']},{metrics['macro_specificity']},{metrics['macro_mcc']}\n")
            f.flush()
    print("Training and validating complete")

<ipython-input-25-6b596e4be73e>:155: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image_path = os.path.join(dataset_path, row[1], row[0])


1262


<ipython-input-25-6b596e4be73e>:179: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  image = torch.tensor(image, dtype=torch.float32)


1762


<ipython-input-25-6b596e4be73e>:188: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image_path = os.path.join(dataset_path, row[1], row[0])


316
(1762, 512)
(316, 512)


model.safetensors:   0%|          | 0.00/175M [00:00<?, ?B/s]

Epoch 1/100, Loss: 1.0283
Epoch 1/100, Loss: 0.6086
Accuracy: 0.7898
Precision: 0.7865
Recall: 0.7898
F1 Score: 0.7863
AUC: 0.9771
AUPR: 0.8978
Specificity per class: [0.9741379310344828, 0.9597989949748744, 0.9333333333333333, 0.952054794520548]
Macro-average Specificity: 0.9548
MCC per class: [0.8517175559495883, 0.8292145851498173, 0.8866845818090036, 0.4027868990023631]
Macro-average MCC: 0.7426
F1 Score (micro): 0.8639
F1 Score (weighted): 0.8640
---------------------------------------------------------------------------------------------------
Epoch 2/100, Loss: 0.5586
Epoch 2/100, Loss: 0.5319
Accuracy: 0.7124
Precision: 0.8602
Recall: 0.7124
F1 Score: 0.7392
AUC: 0.9779
AUPR: 0.9355
Specificity per class: [0.978448275862069, 0.7386934673366834, 0.9822222222222222, 0.9965753424657534]
Macro-average Specificity: 0.9240
MCC per class: [0.5551025765206549, 0.7065517131363608, 0.9620442890220036, 0.5972487489243078]
Macro-average MCC: 0.7052
F1 Score (micro): 0.8038
F1 Score (weight

# coatnet pretrained

In [ ]:
import os
import pandas as pd
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision.models as models
import numpy as np
import timm
import torch
import torch.nn.functional as F
import numpy as np
from torcheval.metrics import MulticlassAccuracy, MulticlassPrecision, MulticlassRecall, MulticlassF1Score, MulticlassAUROC, MulticlassAUPRC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
import random


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)  # For CPU
    torch.cuda.manual_seed(seed)  # For GPU (if used)
    torch.cuda.manual_seed_all(seed)  # For all GPUs
    torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior
    torch.backends.cudnn.benchmark = False


def calculate_metrics(predicted, probabilities, labels, num_classes):
    # Initialize Torcheval metrics
    accuracy_metric = MulticlassAccuracy(num_classes=num_classes, average="macro")
    precision_metric = MulticlassPrecision(num_classes=num_classes, average="macro")
    recall_metric = MulticlassRecall(num_classes=num_classes, average="macro")
    f1_score_metric = MulticlassF1Score(num_classes=num_classes, average="macro")
    auc_metric = MulticlassAUROC(num_classes=num_classes, average="macro")
    aupr_metric = MulticlassAUPRC(num_classes=num_classes, average="macro")

    # Update metrics with predictions
    accuracy_metric.update(predicted, labels)
    precision_metric.update(predicted, labels)
    recall_metric.update(predicted, labels)
    f1_score_metric.update(predicted, labels)
    auc_metric.update(probabilities, labels)
    aupr_metric.update(probabilities, labels)

    # Compute metrics
    accuracy = accuracy_metric.compute()
    precision = precision_metric.compute()
    recall = recall_metric.compute()
    f1_score = f1_score_metric.compute()
    auc = auc_metric.compute()
    aupr = aupr_metric.compute()

    # Specificity calculation
    conf_matrix = confusion_matrix(labels.cpu().numpy(), predicted.cpu().numpy(), labels=range(num_classes))
    specificity_per_class = []

    for i in range(num_classes):
        TP = conf_matrix[i, i]
        FP = conf_matrix[:, i].sum() - TP
        FN = conf_matrix[i, :].sum() - TP
        TN = conf_matrix.sum() - (TP + FP + FN)

        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
        specificity_per_class.append(specificity)

    macro_specificity = np.mean(specificity_per_class)

    # MCC calculation for multi-class classification
    mcc_per_class = []
    for i in range(num_classes):
        # Treat each class as the positive class and others as negative
        binary_pred = (predicted == i).int()
        binary_labels = (labels == i).int()
        mcc = matthews_corrcoef(binary_labels.cpu().numpy(), binary_pred.cpu().numpy())
        mcc_per_class.append(mcc)

    macro_mcc = np.mean(mcc_per_class)

    # Print all metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")
    print(f"AUC: {auc:.4f}")
    print(f"AUPR: {aupr:.4f}")
    print(f"Specificity per class: {specificity_per_class}")
    print(f"Macro-average Specificity: {macro_specificity:.4f}")
    print(f"MCC per class: {mcc_per_class}")
    print(f"Macro-average MCC: {macro_mcc:.4f}")

    f1_score_metric_micro = MulticlassF1Score(num_classes=num_classes, average="micro")
    f1_score_metric_micro.update(predicted, labels)
    f1_score_micro = f1_score_metric_micro.compute()
    print(f"F1 Score (micro): {f1_score_micro:.4f}")

    f1_score_metric_weighted = MulticlassF1Score(num_classes=num_classes, average="weighted")
    f1_score_metric_weighted.update(predicted, labels)
    f1_score_weighted = f1_score_metric_weighted.compute()
    print(f"F1 Score (weighted): {f1_score_weighted:.4f}")


    # Return metrics as a dictionary
    return {
        "accuracy": "{:.4f}".format(accuracy.item()),
        "precision": "{:.4f}".format(precision.item()),
        "recall": "{:.4f}".format(recall.item()),
        "f1_score": "{:.4f}".format(f1_score.item()),
        "auc": "{:.4f}".format(auc.item()),
        "aupr": "{:.4f}".format(aupr.item()),
        "macro_specificity": "{:.4f}".format(macro_specificity.item()),
        "macro_mcc": "{:.4f}".format(macro_mcc.item())
    }


class ImageDataset(Dataset):
    def __init__(self, images, features, labels, transform=None):
        self.images = images
        self.features = features
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        feature = self.features[idx]
        label = self.labels[idx]
        return image, feature, label


# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

set_seed(42)

dataset_path = "baldder_tissue_classification"

train_df = pd.read_csv("train_df.csv")
val_df = pd.read_csv("val_df.csv")

train_images = []
train_labels = []
for idx, row in train_df.iterrows():
    image_path = os.path.join(dataset_path, row[1], row[0])
    image = Image.open(image_path)
    image = transform(image)
    train_images.append(image)
    train_labels.append(row['encoded_label'])

loaded_images = np.load("top_images.npy")
loaded_labels = np.load("top_labels.npy")
print(len(train_images))
for i in range(len(loaded_images)):
    image = loaded_images[i]
    label = loaded_labels[i]

    # Denormalize the image
    image = (image * np.array([0.229, 0.224, 0.225])[:, None, None]) + np.array([0.485, 0.456, 0.406])[:, None, None]
    image = (image * 255).astype(np.uint8)  # Convert to integer type for PIL compatibility

    # Convert NumPy array to PIL image
    image = Image.fromarray(image.transpose(1, 2, 0))  # H x W x C format

    # Apply transformations
    image = transform(image)

    # Convert back to PyTorch tensor (if required)
    image = torch.tensor(image, dtype=torch.float32)
    train_images.append(image)
    train_labels.append(label)

print(len(train_images))

val_images = []
val_labels = []
for idx, row in val_df.iterrows():
    image_path = os.path.join(dataset_path, row[1], row[0])
    image = Image.open(image_path)
    image = transform(image)
    val_images.append(image)
    val_labels.append(row['encoded_label'])

print(len(val_images))

train_features = np.load("features_RFE.npy")[:-val_df.shape[0]]
augmented_features = np.load("features_top_images_RFE.npy")
train_features = np.concatenate((train_features, augmented_features), axis=0)
val_features = np.load("features_RFE.npy")[-val_df.shape[0]:]

print(train_features.shape)
print(val_features.shape)

# Create DataLoaders
train_dataset = ImageDataset(train_images, train_features, train_labels, transform=transform)
val_dataset = ImageDataset(val_images, val_features, val_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.block = timm.create_model('coatnet_0_rw_224.sw_in1k', pretrained=True)
        # self.block.head = nn.Linear(self.block.num_features, 1000)

        self.fc1 = nn.Linear(1000, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)

        self.fc_ann1 = nn.Linear(512, 256)
        self.bn_ann1 = nn.BatchNorm1d(256)
        self.relu_ann1 = nn.ReLU()
        self.dropout_ann1 = nn.Dropout(0.2)
        self.fc_ann2 = nn.Linear(256, 128)
        self.bn_ann2 = nn.BatchNorm1d(128)
        self.relu_ann2 = nn.ReLU()
        self.dropout_ann2 = nn.Dropout(0.2)

        self.fc_comb = nn.Linear(384, 256)
        self.bn_comb = nn.BatchNorm1d(256)
        self.relu_comb = nn.ReLU()
        self.dropout_comb = nn.Dropout(0.2)

        self.fc3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.2)
        self.fc4 = nn.Linear(128, num_classes)

        self.relu = nn.ReLU()


    def forward(self, x, features):
        features = features.float()
        x = self.block(x)
        # print(x.shape)
        x = x.view(-1, 1000)  # Flatten
        x = self.dropout1(self.relu1(self.bn1(self.fc1(x))))
        x = self.dropout2(self.relu2(self.bn2(self.fc2(x))))

        features = self.dropout_ann1(self.relu_ann1(self.bn_ann1(self.fc_ann1(features))))
        features = self.dropout_ann2(self.relu_ann2(self.bn_ann2(self.fc_ann2(features))))

        x = torch.cat((x, features), dim=1)
        x = self.dropout_comb(self.relu_comb(self.bn_comb(self.fc_comb(x))))

        x = self.dropout3(self.relu3(self.bn3(self.fc3(x))))
        x = self.fc4(x)
        return x


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
# Instantiate model, loss function, and optimizer
num_classes = 4
model = SimpleCNN(num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 100
all_metrics = []

best_f1 = -100
best_epoch = 0
best_metric = 0
with open('Results/coatnet_0_rw_224_pretrained.csv', 'w') as f:
    f.write("Epoch,Train Loss,Val Loss,Accuracy,Precision,Recall,F1 Score,AUC,AUPR,Specificity,MCC\n")
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for images, features, labels in train_loader:
            images, features, labels = images.to(device), features.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images, features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        train_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}")

        # Validation loop
        model.eval()
        running_loss = 0.0
        with torch.no_grad():
            y_pred = []
            y_true = []
            y_proba = []
            for images, features, labels in val_loader:
                images, features, labels = images.to(device), features.to(device), labels.to(device)

                outputs = model(images, features)
                loss = criterion(outputs, labels)

                running_loss += loss.item() * images.size(0)

                # Apply softmax to get probabilities
                probabilities = F.softmax(outputs, dim=1)

                # Get the predicted class and corresponding probability
                proba, predicted = torch.max(probabilities, 1)

                # Store results
                y_pred.extend(predicted.cpu().tolist())
                y_true.extend(labels.cpu().tolist())
                y_proba.extend(probabilities.cpu().tolist())

            val_loss = running_loss / len(val_loader.dataset)
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {val_loss:.4f}")
            # Convert lists to tensors
            y_pred = torch.tensor(y_pred)
            y_true = torch.tensor(y_true)
            y_proba = torch.tensor(y_proba)

            metrics = calculate_metrics(y_pred, y_proba, y_true, num_classes)
            all_metrics.append(metrics)

            if best_f1 < float(metrics["f1_score"]):
                best_f1 = float(metrics["f1_score"])
                best_epoch = epoch+1
                best_metric = metrics
            print('---------------------------------------------------------------------------------------------------')

            f.write(f"{epoch+1},{train_loss},{val_loss},{metrics['accuracy']},{metrics['precision']},{metrics['recall']},{metrics['f1_score']},{metrics['auc']},{metrics['aupr']},{metrics['macro_specificity']},{metrics['macro_mcc']}\n")
            f.flush()
    print("Training and validating complete")

<ipython-input-27-1c20469a0934>:155: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image_path = os.path.join(dataset_path, row[1], row[0])


1262


<ipython-input-27-1c20469a0934>:179: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  image = torch.tensor(image, dtype=torch.float32)


1762


<ipython-input-27-1c20469a0934>:188: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image_path = os.path.join(dataset_path, row[1], row[0])


316
(1762, 512)
(316, 512)


model.safetensors:   0%|          | 0.00/110M [00:00<?, ?B/s]

Epoch 1/100, Loss: 1.0680
Epoch 1/100, Loss: 0.8594
Accuracy: 0.5511
Precision: 0.5403
Recall: 0.5511
F1 Score: 0.4919
AUC: 0.8937
AUPR: 0.7409
Specificity per class: [0.728448275862069, 0.9949748743718593, 0.7866666666666666, 1.0]
Macro-average Specificity: 0.8775
MCC per class: [0.512826046448358, 0.4984767351208064, 0.7176433587846222, 0.0]
Macro-average MCC: 0.4322
F1 Score (micro): 0.6456
F1 Score (weighted): 0.5954
---------------------------------------------------------------------------------------------------
Epoch 2/100, Loss: 0.6776
Epoch 2/100, Loss: 0.3674
Accuracy: 0.7640
Precision: 0.9192
Recall: 0.7640
F1 Score: 0.7760
AUC: 0.9866
AUPR: 0.9455
Specificity per class: [0.9267241379310345, 0.9396984924623115, 0.9822222222222222, 1.0]
Macro-average Specificity: 0.9622
MCC per class: [0.7969989389292474, 0.8942135564132069, 0.9620442890220036, 0.4422731886731076]
Macro-average MCC: 0.7739
F1 Score (micro): 0.8956
F1 Score (weighted): 0.8787
---------------------------------